In [18]:
# Install needed python modules
!pip install click | grep -v 'already satisfied'
!pip install numpy | grep -v 'already satisfied'
!pip install requests | grep -v 'already satisfied'
!pip install Pillow | grep -v 'already satisfied'
!pip install torch torchvision torchaudio | grep -v 'already satisfied'
!pip install pandas | grep -v 'already satisfied'
!pip install scipy | grep -v 'already satisfied'
print("All dependencies installed")


All dependencies installed


In [1]:
# Import modules
import os
import re
from typing import List, Optional, Tuple, Union

import click
import dnnlib
import numpy as np
import PIL.Image
import torch

import legacy
from stablehash_dataset import StableHashDataset
from torch.utils.data import DataLoader

print("All modules imported")

All modules imported


In [2]:
# Set Nvidia Network model 

network_pkl = "/home/ubuntu/nvidia-model/stylegan3-r-ffhq-1024x1024.pkl"

# Load training dataset and create iterator
data_dir = "/home/ubuntu/stablehash-reconstruct/output"
data_csv = "/home/ubuntu/stablehash-reconstruct/training_data.csv"
train_data = StableHashDataset(data_dir, data_csv)
train_dataloader = DataLoader(train_data, batch_size=4, shuffle=True)
#stablehash, original_img = next(iter(train_dataloader))

print("All variables set")

All variables set


In [3]:
def make_transform(translate: Tuple[float,float], angle: float):
    m = np.eye(3)
    s = np.sin(angle/360.0*np.pi*2)
    c = np.cos(angle/360.0*np.pi*2)
    m[0][0] = c
    m[0][1] = s
    m[0][2] = translate[0]
    m[1][0] = -s
    m[1][1] = c
    m[1][2] = translate[1]
    return m


def generate_images():
    
    seeds = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    truncation_psi = 0.75
    noise_mode = "const"
    outdir  = "/home/ubuntu/stablehash-reconstruct/output"
    translate = 0,0
    rotate = 0
    class_idx = None
    
    # 1) Load pre-trained Generator
    print('Loading networks from "%s"...' % network_pkl)
    device = torch.device('cuda')
    with dnnlib.util.open_url(network_pkl) as f:
        G = legacy.load_network_pkl(f)['G_ema'].to(device) # type: ignore
    print("Network loaded")
    
    # 2) Define mapping and synthesis network (freeze)
    mapping_n = G.mapping
    synthesis_n = G.synthesis
    for mapping_param in mapping_n.parameters(): # set mapping network params to trainable
        mapping_param.requires_grad = True
    for synthesis_param in synthesis_n.parameters(): # set synthesis params to frozen
        synthesis_param.requires_grad = False
    print("Defined boolean mapping")
    
    # 3) Define optimizer --> in this case we use ADAM
    optim = torch.optim.Adam(mapping_n.parameters(), lr=0.0025, betas=[0,0.99], eps=1e-8)
    print("Optimizer defined")
    
    # 4) Define simple MSE loss function
    mse_loss = torch.nn.MSELoss(reduction="mean")

    for i in range(1000):
        stablehash, orig_img = next(iter(train_dataloader))
        w = G.mapping(torch.flip(stablehash, [1]), class_idx) # remove flip function --> only for testing
        gen_img = G.synthesis(w)
        gen_img = (gen_img * 127.5 + 128).clamp(0, 255).float()
        loss_value = mse_loss(gen_img, orig_img)
        loss_value.backward()
        optim.step() #gradient descent
        print(loss_value)
    print("MSE loss function defined")
        


In [4]:
 generate_images() 

Loading networks from "/home/ubuntu/nvidia-model/stylegan3-r-ffhq-1024x1024.pkl"...
Network loaded
Defined boolean mapping
Optimizer defined
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "filtered_lrelu_plugin"... Done.


/home/ubuntu/environments/my_env/lib/python3.8/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([4, 3, 160, 160])) that is different to the input size (torch.Size([4, 3, 1024, 1024])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (1024) must match the size of tensor b (160) at non-singleton dimension 3